### Goals  
    1. Load the clinical variables
    2. Process the data as a pandas dataframe
    3. Research SurvhShap, speficially what kind of data it expects.
    4. Create m parameter-cuts of the data as equidistant
    5. Implement an event-sorting algorithm based on the cuts and events.
    6. Finish processing the data so its ready for training
    7. Setup a network alongside the binary-cross-loss (sigmoid on the network output).
    8. Implement a function to create survival function for the chosen slices (0, m)
    9. Plot survival for five individuals
    10. plot accuracy
    11. attempt C-index or Brier score on the network outputs.

In [34]:
import pandas as pd
import openpyxl

In [35]:
#1. Load the clinical data
data = pd.read_excel("Colorectal-Liver-Metastases-Clinical-data-April-2023.xlsx")

/home/anders/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [36]:
#2. Process the data as pandas dataframe
#Dropping the DFS as I'm going to be modeling ordinary survival, without competing interests
surv_data = data[["vital_status", "overall_survival_months"]].copy()
identity_data = data[["Patient-ID", "De-identify Scout Name"]].copy()
data.drop(columns=["vital_status", "overall_survival_months", "Patient-ID", "De-identify Scout Name", "months_to_DFS_progression", "vital_status_DFS", "months_to_liver_DFS_progression", "vital_status_liver_DFS", "relevant_notes", "progression_or_recurrence", "progression_or_recurrence_liveronly"], inplace=True)

"""
train_set = data
val_set = train_set.sample(frac=0.2)
train_set.drop(val_set.index, inplace=True)
"""


'\ntrain_set = data\nval_set = train_set.sample(frac=0.2)\ntrain_set.drop(val_set.index, inplace=True)\n'

In [37]:
from sksurv.util import Surv

In [38]:
#Do not run this cell more than once!
#Creating Train and validation sets (using validation as test and validation. )
X_train = data
X_val = X_train.sample(frac=0.2)
X_train.drop(X_val.index, inplace=True)
surv_data_train = surv_data.drop(X_val.index)
surv_data_val = surv_data.drop(X_train.index)
Y_train = Surv.from_dataframe("vital_status", "overall_survival_months", surv_data_train)
Y_val   = Surv.from_dataframe("vital_status", "overall_survival_months", surv_data_val)
print(surv_data.shape)
print(Y_train.shape, Y_val.shape)
print(X_train.shape, X_val.shape)


(197, 2)
(158,) (39,)
(158, 25) (39, 25)


#### 3. Research what types of the data the SurvShap package expects as input

In the original tutorial notebook the data for training and validation go through the same process as the cell above, where the X_train containing the covariates are pandas dataframes and the Y_train have gone through the Surv() package. 

SurvShap also expects some function that returns the Survival equation for all the time-steps based on the network output, and this has to come on the form: 

## So models like the LogisticHazard use sigmoid on the output to essentially output hazard functions for each of the [0,m] times, and then uses binary cross entropy loss to train the network, but what does it train against? No dataset contains h or S values for comparison, so how do you steer this thing?  

The answer is that it assumes the ideal network output is 1 or 0 for the appropriate event_duration.  

so the network could output: 0,0,0,0,1,0,0,1  
which could be compared to:  0,0,1,0,0,0,0,1

using the BCE with logits loss.

In fact I could probably import the loss and the surv and hazard functions from the class to do this. 